<font style='font-size:1.5em'>**👨🏻‍🏫 Week 04 lecture – Web Scraping** </font>

<font style='font-size:1.2em'>DS105W – Data for Data Science</font>

**AUTHORS:**  Dr. [Jon Cardoso-Silva](https://jonjoncardoso.github.io)

**DEPARTMENT:** [LSE Data Science Institute](https://lse.ac.uk/dsi)

**OBJECTIVE**: Learn how to collect data from the Web using Python packages

**LAST REVISION:** 8 February 2024




--- 

# Part I: ⚙️ The setup

You will need to install the requests and Scrapy packages in order to complete this lab. I will assume you have configured the virtual environment for this course as follows. 



Open the terminal (directly from within VS Code will be easier) and run each of the following commands:


```bash
pip install pandas requests scrapy
```


In [ ]:
import requests               # This is how we access the web
import pandas as pd           # This is how we work with data frames

from pprint import pprint     # Print things in a pretty way
from scrapy import Selector   # This is how we parse HTML


# Part II: Requesting a web page


You might have heard of [CIVICA](https://www.civica.eu/who-we-are/about-civica/) before. It is a body that unites several European universities to collaborate in the areas of social sciences, humanities, business and public policy. CIVICA hosts [Data Science Seminar series](https://socialdatascience.network/index.html#schedule) that might be of interest to you. Today we will collect information on some of the seminars. Maybe you can use it in the future! 

**Our main task** is to create a 🐼 pandas data frame that would contain:

1. names of the seminars 
2. names of speakers of those seminars
3. dates of the seminars
4. bios of the speakers from each individual event 


## 2.1. Request a website


In [ ]:
# This is the address of the website we want to scrape
my_url = 'https://socialdatascience.network/index.html#schedule'

# We set a GET request to the website
response = requests.get(my_url)

# What is the response code?
response

**📜 Other possible responses**

The response code is standard way of communicating the status of a request. There are many other possible responses:

- **200** OK
- **204** No Content
- **400** Bad Request
- **401** Unauthorized
- **402** Payment Required
- **403** Forbidden
- **404** Not Found
- **500** Internal Server Error
- **502** Bad Gateway

🗣️ **CLASSROOM DISCUSSION:** Have you ever encountered any of these responses when browsing the Web on your browser? Where? What did you do about it?


You can find a full list [here](https://developer.mozilla.org/en-US/docs/Web/HTTP/Status).

## 2.2. A closer look at the response

What else is stored in the `response` object?

In [ ]:
# The vars function returns all attributes of an object, along with their values
# You will see that it is essentially just a dictionary
vars(response)

🗣️ **CLASSROOM DISCUSSION:**

You have already looked at `response.status_code`. But what do you think the following attributes of the `response` object are?

- `response.headers`
- `response.cookies`
- `response.content`

Feel free to open a new chunk of code below and explore these attributes.

But encoding is not the only **metadata** we can get from the response. Let's take a look at all the headers:

In [ ]:
# Headers are metadata about the response
pprint(response.headers)

We could choose to manipulate the headers above as a `pd.Series`:

In [ ]:
pd.Series(response.headers)

Let me know you what is in the object `response` by printing it.

In [ ]:
pprint(response.text)

The code chunk above makes sense here because I want to show you how to inspect objects when in **prototype mode**. However, whenever you are writing in a Jupyter Notebook to report to someone (say, when submitting your assignment), you should remove code chunks that produce a lot of unnecessary output.


💡 **A DETAIL THAT SEEMS INSIGNIFICANT BUT THAT IS EXTREMELY IMPORTANT**: 
- If you are on Mac or Linux, you will find that the break line character is `\n`. 
- If you are on Windows, you will find that the break line character is `\r\n`. 
- Windows uses two characters to break lines, while Mac and Linux use only one. 
- This is a common source of errors when working with with text files in two different OS. (For example: you use Mac and collaborate with someone who uses Windows.)

How many characters are there in the `response.text`?

In [ ]:
len(response.text)

Not very useful to treat is as pure string, right? We need to find a better way to parse this data.


## 2.3. Parsing HTML

The Scrapy Selector package is a Python library for extracting data from HTML and XML documents. It uses CSS or XPath selectors for data extraction making it a powerful tool for web scraping. It is often an essential part of the Scrapy framework but can also be used independently.

When you feed HTML text to the Scrapy Selector, it processes the HTML and preserves it in a particular **object**. This object allows you to access parts of the HTML using Python's common dot notation in combination with the CSS syntax. If, for instance, you want to fetch the title of the page, you might use `selector.css('title')`.


In [ ]:
# parse the HTML code using Scrapy Selector
sel = Selector(text=response.text)

💡 Note: I was only able to call `Selector()` directly because I had already imported it at the top of the notebook. Scroll up to see it. If I hadn't, the code above would have thrown an error.

**Check `sel.get()` to see the full HTML document**

This has the same effect as `response.text`.

In [ ]:
sel.get()

**HTML documents usually have a \<header\> tag:**

(⚠️ not to be confused with the HTTP header we saw with `response.headers`)

In [ ]:
sel.css('header')

There is also usually a `<body>` tag, which contains the main content of the page:

In [ ]:
sel.css('body')

🔑 **Takeaway of the output above:**

- The output is a list, as indicated by the square brackets. 
- HTML pages only have one `<body>` tag, so this list contains a single element, which is an object of the class Selector.

What if I want to look at the content of the `<body>` tag?

In [ ]:
pprint(sel.css('body').get())

**Are there any `<h1>` tags in this page?**

In [ ]:
sel.css('h1').get()

What about `<h2>` tags?

In [ ]:
sel.css('h2').getall()

If you care just about the **first** `<h2>` tag, you can use the `.get()` method instead of `.getall()`:

In [ ]:
sel.css("h2").get()

**How to get the text from a tag:**

In [ ]:
sel.css("h2 ::text").get()

**How to get the text of tags returned by the `.css()` method?**

You can also use `::text` on each tag element within the CSS selector returned by the `css()` method.


In [ ]:
# Pure Python way
all_h2_tags = sel.css("h2 ::text").getall()
all_h2_texts = []

for tag in all_h2_tags:
    all_h2_texts.append(tag)

all_h2_texts

**Consider using [list comprehension](https://www.w3schools.com/python/python_lists_comprehension.asp) for a cleaner code:**

In [ ]:
# one-liner way
all_h2_texts = [tag.get() for tag in sel.css("h2 ::text")]
all_h2_texts

💡 **IMPORTANT TIPS:**

- Make it a habit in the next couple of weeks to every now and then, right-click on a webpage and select "Inspect" (or "Inspect Element") to explore how the HTML is structured. This will help you understand how to use CSS selectors to extract the data you need.
- Tag names and ` ::text` are just the tip of the iceberg. Read about other CSS selectors [here](https://www.w3schools.com/cssref/css_selectors.asp).
- Bookmark the [Scrapy Selectors documentation page](https://docs.scrapy.org/en/latest/topics/selectors.html) and revisit it whenever you need to. Practice using different CSS selectors to extract data from the HTML.

# Part III: Your turn!

Let's make this a dynamic and interactive lecture.


🎯 **ACTION POINTS**

1. Go to the [Data Science Seminar series](https://socialdatascience.network/index.html#schedule) website and inspect the page (mouse right-click + Inspect) and find the way to the name of the first event on the page. 

3. Write down the "directions" inside the HTML file to reach the event title. For example, maybe you will find that:

    > _The first event title is inside a \<html\> ➡️ \<div\> ➡️ \<div\> ➡️ \<h3\> tag_.

    Write it in the markdown cell below:

_Delete this line and write your answer here_

4. Now, use the skill that you have just learned to scrape the names of ALL events. Save them all to a list.

In [ ]:
# Delete this line and replace it with your code

5. Do the same with the dates of the events and speaker names and save them to separate lists. 



In [ ]:
# Delete this line and replace it with your code

6. Combine all of these the lists into a pandas data frame and save it to a CSV file.

In [ ]:
# Delete this line and replace it with your code

7. Double-check that the CSV file was created correctly by opening it using pandas. Then convert the columns to appropriate data types.

In [ ]:
# Delete this line and replace it with your code

# Part IV: Why convert the scraped data to a data frame? 

If time allows, we will play around with the data frame in pandas to learn a few other tricks.